In [1]:
from gazpacho import get, Soup

In [2]:
url = 'https://www.capfriendly.com/'
html = get(url)

In [3]:
soup = Soup(html)

In [4]:
table = soup.find('table', {'id': 'ich'})

In [5]:
str(table)[:100]

'<table id="ich" class="sortablex tblcf index tbl sortable"><thead><tr class="column_head"><th class='

In [6]:
trs = table.find('tr', {'class': 'tmx'})

In [7]:
tr = trs[0]

In [8]:
tr.find('a', mode='first').text

'Arizona Coyotes'

In [9]:
tr.find('td', {'data-label': 'PROJECTED CAP HIT'}, strict=True).text

'$84,973,671'

In [10]:
def parse_tr(tr):
    team = tr.find('a', mode='first').text
    cap = tr.find('td', {'data-label': 'PROJECTED CAP HIT'}, strict=True).text
    cap = float(cap.replace(',', '').replace('$', ''))
    return team, cap

In [11]:
cap_hits = [parse_tr(tr) for tr in trs]

In [12]:
url = 'https://www.hockey-reference.com/friv/playoff_prob.fcgi'
html = get(url)

In [13]:
soup = Soup(html)

In [14]:
import pandas as pd

In [15]:
east = pd.read_html(str(soup.find('table')[0]))[0]
west = pd.read_html(str(soup.find('table')[1]))[0]

In [16]:
df = pd.concat([east, west])[['Team', 'W']].reset_index(drop=True)
df['W'] = df['W'].apply(pd.to_numeric, errors='coerce')
wins = df.dropna()

In [17]:
cap_hits = pd.DataFrame(cap_hits, columns=['Team', 'spend'])
df = pd.merge(wins, cap_hits, on='Team', how='left')
df['mpw'] = round(df['spend'] / df['W'] / 1_000_000, 2)
df.sort_values('mpw', ascending=True)

,Team,W,spend,mpw
9,New York Islanders,48.3,75517168.0,1.56
17,Colorado Avalanche,46.6,74711199.0,1.60
8,Washington Capitals,49.8,81585315.0,1.64
10,Pittsburgh Penguins,47.5,80725216.0,1.70
11,Carolina Hurricanes,46.2,79972980.0,1.73
16,St. Louis Blues,48.1,83126001.0,1.73
1,Tampa Bay Lightning,44.8,79252023.0,1.77
0,Boston Bruins,46.7,82612311.0,1.77
19,Winnipeg Jets,42.9,76537729.0,1.78
18,Dallas Stars,43.9,81825021.0,1.86


In [18]:
from IPython.display import HTML
HTML('<img src="https://media.giphy.com/media/oOX5qIDkzDjeo/giphy.gif">')

### Saving results

In [19]:
df.to_csv('data/mpw.csv', index=False)

In [20]:
df.head()

,Team,W,spend,mpw
0,Boston Bruins,46.7,82612311.0,1.77
1,Tampa Bay Lightning,44.8,79252023.0,1.77
2,Toronto Maple Leafs,43.7,95109190.0,2.18
3,Florida Panthers,42.1,81398277.0,1.93
4,Montreal Canadiens,39.1,77290175.0,1.98


In [21]:
df['date_fetched'] = pd.Timestamp('today')

In [22]:
df.head()

,Team,W,spend,mpw,date_fetched
0,Boston Bruins,46.7,82612311.0,1.77,2020-01-02 00:17:50.488401
1,Tampa Bay Lightning,44.8,79252023.0,1.77,2020-01-02 00:17:50.488401
2,Toronto Maple Leafs,43.7,95109190.0,2.18,2020-01-02 00:17:50.488401
3,Florida Panthers,42.1,81398277.0,1.93,2020-01-02 00:17:50.488401
4,Montreal Canadiens,39.1,77290175.0,1.98,2020-01-02 00:17:50.488401


In [23]:
import sqlite3

con = sqlite3.connect('data/mpw.db')

df.to_sql('teams', con, index=False, if_exists='append')

In [24]:
pd.read_sql('''
    select 
    * 
    from teams 
    where mpw > 2 and W < 41
    order by mpw desc
''', con)

,Team,W,spend,mpw,date_fetched
0,Detroit Red Wings,24.7,80103733.0,3.24,2020-01-02 00:17:50.488401
1,San Jose Sharks,34.7,80797623.0,2.33,2020-01-02 00:17:50.488401
2,Buffalo Sabres,37.0,84299820.0,2.28,2020-01-02 00:17:50.488401
3,New Jersey Devils,32.9,75078373.0,2.28,2020-01-02 00:17:50.488401
4,Anaheim Ducks,35.3,79318385.0,2.25,2020-01-02 00:17:50.488401
5,Los Angeles Kings,35.2,77075319.0,2.19,2020-01-02 00:17:50.488401
6,Chicago Blackhawks,38.1,82903043.0,2.18,2020-01-02 00:17:50.488401
7,Ottawa Senators,35.4,75277328.0,2.13,2020-01-02 00:17:50.488401
8,Calgary Flames,39.6,82012291.0,2.07,2020-01-02 00:17:50.488401
9,Edmonton Oilers,40.1,80590618.0,2.01,2020-01-02 00:17:50.488401
